In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('bmh')

In [7]:
df = (pd.read_json('data.json', orient = 'index')
        .reset_index()
        .rename(columns = {'index': 'date'})
        .melt(id_vars = 'date',
              var_name = 'exercise',
              value_name = 'load',
              value_vars = ['squat', 'deadlift', 'overhead press'])
        .dropna()
        .sort_values(by = 'date')
        .reset_index())

df.tail()

,index,date,exercise,load
101,104,2022-11-23,deadlift,170.0
102,48,2022-11-27,squat,145.0
103,154,2022-11-27,overhead press,65.0
104,105,2022-11-28,deadlift,170.0
105,158,2022-11-28,overhead press,65.0


In [24]:
def plot_pbc(df, data_var, window = 20, signal_window = 8, ax = None, display_df = False, **kwargs):
    df = df.copy()
    
    df['moving_average'] = (df[data_var].sort_index(ascending = False)
                                        .rolling(window, min_periods = 1)
                                        .mean())
    
    df['moving_range'] = (df[data_var].diff(-1)
                                      .abs()
                                      .sort_index(ascending = False)
                                      .rolling(window, min_periods = 1)
                                      .mean())

    df['process_average'] = df['moving_average']
    df['process_range'] = df['moving_range']
    df['signal'] = None
    df['signal_min'] = None
    df['signal_max'] = None
    df['signal_above_average'] = None
    df['signal_below_average'] = None
    display(df)

    n_rows = len(df)
    previous_signal_id = 0

    for row in np.arange(n_rows):
        first_row = row == 0
        sufficient_rows_left = n_rows - row >= window

        signal_start_id = np.max([8, row - signal_window])
        print([signal_start_id, row])

        df['signal_min'].iat[row] = df[data_var][signal_start_id:row].min()
        df['signal_max'].iat[row] = df[data_var][signal_start_id:row].max()

        df['signal_above_average'].iat[row] = (df['signal_min'][row] > df['process_average'][row - 1])
        df['signal_below_average'].iat[row] = (df['signal_max'][row] < df['process_average'][row - 1])

        signal_open = (first_row) | (row >= previous_signal_id + window)
        signal = (signal_open) & (sufficient_rows_left) & (first_row | df['signal_above_average'][row] | df['signal_below_average'][row])
        df['signal'].iat[row] = signal
        
        df['process_average'].iat[row] =  df['process_average'][row - 1]
        df['process_range'].iat[row] =  df['process_range'][row - 1]

        if signal:
            previous_signal_id = row
            df['process_average'].iat[row] =  df['moving_average'][row]
            df['process_range'].iat[row] =  df['moving_range'][row]
        else:
            df['process_average'].iat[row] =  df['process_average'][row - 1]
            df['process_range'].iat[row] =  df['process_range'][row - 1]

    zones = 3

    for i in np.arange(zones):
        offset = df['process_range']*(i + 1)/1.128
        df[f'lower_limit_{i}'] = df['process_average'] - offset
        df[f'upper_limit_{i}'] = df['process_average'] + offset

    if display_df:
        display(df)

    if ax is None:
        ax = plt.gca()

    ax.scatter(df.index, df[data_var], marker = '.', alpha = 0.3, color = 'slategray', zorder = 3)
    ax.plot(df.index, df['process_average'], linestyle = '--', color = 'slategray', zorder = 3, linewidth = 1)

    colors = {'lower': ['#f7f7f7', '#f4a582', '#ca0020'],
              'upper': ['#f7f7f7', '#92c5de', '#0571b0']}

    for i in np.arange(zones):
        prev_i = np.max([0, i - 1])

        if i > 0:
            for level in ['lower', 'upper']:
                ax.fill_between(df.index, df[f'{level}_limit_{i}'], df[f'{level}_limit_{prev_i}'], alpha = 0.5, color = colors[level][i], zorder = 1, label = f'Zone {i + 1} ({level})', linewidth = 0)

        if i == 0:
            ax.fill_between(df.index, df[f'lower_limit_{i}'], df[f'upper_limit_{prev_i}'], alpha = 0.5, color = colors['lower'][i], zorder = 1, label = 'Zone 1', linewidth = 0)

    return ax

In [25]:
plot_pbc(df[df['exercise'] == 'squat'], 'load')

,index,date,exercise,load,moving_average,moving_range,process_average,process_range,signal,signal_min,signal_max,signal_above_average,signal_below_average
0,0,2022-09-17,squat,135.0,134.000000,3.000000,134.000000,3.000000,None,None,None,None,None
2,1,2022-09-18,squat,130.0,134.000000,3.000000,134.000000,3.000000,None,None,None,None,None
4,2,2022-09-19,squat,135.0,134.500000,2.750000,134.500000,2.750000,None,None,None,None,None
6,3,2022-09-20,squat,130.0,134.750000,2.500000,134.750000,2.500000,None,None,None,None,None
8,4,2022-09-21,squat,135.0,135.250000,2.250000,135.250000,2.250000,None,None,None,None,None
10,5,2022-09-22,squat,135.0,135.500000,2.375000,135.500000,2.375000,None,None,None,None,None
13,6,2022-09-23,squat,135.0,135.875000,2.500000,135.875000,2.500000,None,None,None,None,None
15,7,2022-09-24,squat,135.0,136.375000,2.625000,136.375000,2.625000,None,None,None,None,None
17,8,2022-09-25,squat,130.0,136.750000,2.500000,136.750000,2.500000,None,None,None,None,None
19,9,2022-09-26,squat,135.0,137.500000,2.750000,137.500000,2.750000,None,None,None,None,None


[8, 0]


KeyError: -1